In [13]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [14]:
# Cargar la capa temática
natalidad = "/Users/jesussono/PycharmProjects/flask-dash-board/data/natalidad2.geojson"
map_data = gpd.read_file(natalidad)
map_data.head()

,NAME_1,NAME_2,CC_2,NAT2018,geometry
0,Andalucía,Almería,04,10.48,"MULTIPOLYGON (((-3.03042 35.94236, -3.03375 35..."
1,Andalucía,Cádiz,11,8.11,"MULTIPOLYGON (((-6.21958 36.38110, -6.21958 36..."
2,Andalucía,Córdoba,14,7.90,"MULTIPOLYGON (((-5.04854 37.63690, -5.02637 37..."
3,Andalucía,Granada,18,7.78,"POLYGON ((-3.35014 36.72952, -3.77292 36.73125..."
4,Andalucía,Huelva,21,7.57,"MULTIPOLYGON (((-6.83648 37.11547, -6.83458 37..."


In [15]:
map_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
NAME_1      50 non-null object
NAME_2      50 non-null object
CC_2        50 non-null object
NAT2018     50 non-null float64
geometry    50 non-null geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 2.1+ KB


In [16]:
map_data.head()

,NAME_1,NAME_2,CC_2,NAT2018,geometry
0,Andalucía,Almería,04,10.48,"MULTIPOLYGON (((-3.03042 35.94236, -3.03375 35..."
1,Andalucía,Cádiz,11,8.11,"MULTIPOLYGON (((-6.21958 36.38110, -6.21958 36..."
2,Andalucía,Córdoba,14,7.90,"MULTIPOLYGON (((-5.04854 37.63690, -5.02637 37..."
3,Andalucía,Granada,18,7.78,"POLYGON ((-3.35014 36.72952, -3.77292 36.73125..."
4,Andalucía,Huelva,21,7.57,"MULTIPOLYGON (((-6.83648 37.11547, -6.83458 37..."


In [17]:
df = map_data.copy()

In [18]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
NAME_1      50 non-null object
NAME_2      50 non-null object
CC_2        50 non-null object
NAT2018     50 non-null float64
geometry    50 non-null geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 2.1+ KB


In [19]:
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, String, JSON, Float
from geoalchemy2 import Geometry, WKTElement

engine = create_engine('postgresql://localhost/portfolio_db', echo=False)

In [20]:
df.head()

,NAME_1,NAME_2,CC_2,NAT2018,geometry
0,Andalucía,Almería,04,10.48,"MULTIPOLYGON (((-3.03042 35.94236, -3.03375 35..."
1,Andalucía,Cádiz,11,8.11,"MULTIPOLYGON (((-6.21958 36.38110, -6.21958 36..."
2,Andalucía,Córdoba,14,7.90,"MULTIPOLYGON (((-5.04854 37.63690, -5.02637 37..."
3,Andalucía,Granada,18,7.78,"POLYGON ((-3.35014 36.72952, -3.77292 36.73125..."
4,Andalucía,Huelva,21,7.57,"MULTIPOLYGON (((-6.83648 37.11547, -6.83458 37..."


In [21]:
df['geometry'] = df['geometry'].apply(lambda x: str(WKTElement(x.wkt)))

In [22]:
df['id'] = np.arange(len(df))

In [27]:
df.columns = ['name_1', 'name_2', 'cc_2', 'nat2018', 'geometry', 'id']

In [28]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
name_1      50 non-null object
name_2      50 non-null object
cc_2        50 non-null object
nat2018     50 non-null float64
geometry    50 non-null object
id          50 non-null int64
dtypes: float64(1), int64(1), object(4)
memory usage: 2.5+ KB


In [30]:
from sqlalchemy.types import Float, String, Integer
df.to_sql('geodata', con=engine, index=False, if_exists = 'append',
          dtype={"id": Integer(),
                 "name_1": String(),
                "name_2": String(),
                "cc_2": String(),
                "nat2018": Float(),
                "geometry": String()})

In [ ]:
df_out = pd.read_sql_table('geodata', engine) 

In [ ]:
from shapely import wkt
import geopandas as gpd

In [ ]:
df_out.head()

In [ ]:
df_out['geometry'] = df_out['geometry'].apply(wkt.loads)
my_geo_df = gpd.GeoDataFrame(df_out, geometry='geometry')

In [ ]:
my_geo_df.info()

In [ ]:
# Control del tamaño de la figura del mapa
fig, ax = plt.subplots(figsize=(10, 10))
 
# Control del título y los ejes
ax.set_title('Natalidad por Provincias en España, 2018', 
             pad = 20, 
             fontdict={'fontsize':20, 'color': '#4873ab'})
ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')
 
# Mostrar el mapa finalizado
my_geo_df.plot(column='NAT2018', cmap='plasma', ax=ax, zorder=5)